In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('Life Expectancy Data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10   BMI                             2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio               

In [2]:
# Xóa các dòng có giá trị rỗng
df.dropna(inplace=True)

data = df['Country']

# Xóa các cột không cần thiết
df.drop(['Country', 'Year'], axis=1, inplace=True)

# Chuyển Status thành số
df['Status'] = df['Status'].map({'Developing': 1, 'Developed': 0})

# Chuyển đổi các biến số thành dạng số thực
df = df.astype(float)

# Chia dữ liệu thành tập train và tập test
X = df.drop('Life expectancy ', axis=1)
y = df['Life expectancy ']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình Linear Regression trên tập huấn luyện
model = LinearRegression()
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

In [3]:
# 1. Tính các sai số MAE, MSE, R^2
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)
print("R^2:", r2)

MAE: 2.8131951604837315
MSE: 13.457911853390346
R^2: 0.8105119533384194


In [4]:
# 2. So sánh các giá trị sai số trước và sau khi chuẩn hóa dữ liệu

from sklearn.preprocessing import MinMaxScaler

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled = scaler.fit_transform(X)

# Huấn luyện mô hình với dữ liệu đã chuẩn hóa
model_scaled = LinearRegression()
model_scaled.fit(X_train_scaled, y_train)

# Dự đoán trên tập kiểm tra đã chuẩn hóa
y_pred_scaled = model_scaled.predict(X_test_scaled)

# Tính các chỉ số sai số
mae_scaled = mean_absolute_error(y_test, y_pred_scaled)
mse_scaled = mean_squared_error(y_test, y_pred_scaled)
r2_scaled = r2_score(y_test, y_pred_scaled)

print("MAE before scale:", mae)
print("Scaled MAE:      ", mae_scaled, end='\n\n')

print("MSE before scale:", mse)
print("Scaled MSE:      ", mse_scaled, end='\n\n')

print("R^2 before scale:", r2)
print("Scaled R^2:      ", r2_scaled)

MAE before scale: 2.8131951604837315
Scaled MAE:       2.813195160479853

MSE before scale: 13.457911853390346
Scaled MSE:       13.45791185333952

R^2 before scale: 0.8105119533384194
Scaled R^2:       0.8105119533391351


In [5]:
# 3. Sử dụng kỹ thuật cross validation để tối ưu hóa các chỉ số

kf = KFold(n_splits=5, shuffle=True, random_state=42)
# n_splits=5: Số lượng fold (tập con)
# shuffle=True: Lựa chọn ngẫu nhiên các mẫu dữ liệu

mae_cv = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=kf)
mse_cv = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=kf)
r2_cv = cross_val_score(model, X_train, y_train, scoring='r2', cv=kf)

print("MAE (cross validation):", abs(mae_cv.mean()))
print("MSE (cross validation):", abs(mse_cv.mean()))
print("R^2 (cross validation):", abs(r2_cv.mean()))

MAE (cross validation): 2.783509929948295
MSE (cross validation): 13.023264103716475
R^2 (cross validation): 0.8333220382874377
